# Data Synchronization

This notebook is an implementation of a data synchronization structure for interpolating blob positions in order to solve asynchrony issues. 

The goal is to implement and analyse a method of synchronizing the data received by the clients in a standard time vector. For this:
1. Make *CoppeliaSim's* clients that detects and sends blob coordinates detected in Vision Sensors' images;
2. Receive the messages asynchronously;
3. For each client, interpolate in time the detected blob positions;
4. Show the interpolated data of each camera for further analysis. 

---

In [1]:
# Importing modules...
import numpy as np
import scipy as sp
from scipy.interpolate import CubicSpline
from scipy.optimize import linear_sum_assignment
import cv2
import socket

import sys
sys.path.append('../..') # Go back to base directory

from modules.vision.camera import Camera
from modules.integration.client import Client
from modules.integration.coppeliasim.server import CoppeliaSim_Server

# Instanciating `Server` and `Client` Structures

To wrap the information of all clients and mediate the communication between this notebook and *CoppeliaSim*, a `Server` object will be instanciated. 

The `Client` objects will be generated by their camera model and will represent their respective twin in the simulation.

For the `Server` instanciation, the following parameters must be given:
- Server address;
- *CoppeliaSim's* simulation Controller address;
- List containing all the clients present in the scene.

---

In [2]:
n_clients = 4 # Number of clients in the arena
clients = []  # Clients list

# Object matrix of Camera 0
base_matrix = np.array([[-7.07106781e-01,  5.00000000e-01, -5.00000000e-01, 2.50000000e+00],
                        [ 7.07106781e-01,  5.00000000e-01, -5.00000000e-01, 2.50000000e+00],
                        [ 1.46327395e-13, -7.07106781e-01, -7.07106781e-01, 2.50000000e+00]])

# Create clients
for ID in range(n_clients):
    # Spread all cameras uniformely in a circle around the arena
    R = np.array(sp.spatial.transform.Rotation.from_euler('z', (360 / n_clients) * ID, degrees=True).as_matrix())
    object_matrix = R @ base_matrix

    # Generate associated camera model
    camera = (Camera(# Intrinsic Parameters
                     resolution=(720,720), 
                     fov_degrees=60.0,     
    
                     # Extrinsic Parameters
                     object_matrix=object_matrix,
    
                     # Rational Lens Distortion Model
                     # distortion_model='rational',
                     # distortion_coefficients=np.array([0.014, -0.003, -0.0002, -0.000003, 0.0009, 0.05, -0.007, 0.0017]), 
                    
                     # Fisheye Lens Distortion Model
                     distortion_model='fisheye',
                     distortion_coefficients=np.array([0.395, 0.633, -2.417, 2.110]),
    
                     # Image Noise Model
                     snr_dB=13
                     ))
    
    clients.append(Client(camera=camera))

In [3]:
# Create server
server = CoppeliaSim_Server(clients=clients,
                            server_address=('127.0.0.1', 8888),
                            controller_address=('127.0.0.1', 7777))

# Requesting Scene

A **Scene Request** will send to *CoppeliaSim* the data necessary to instanciate the simulated clients' twins in the childscripts. The scene request can be used to reset the client's data from the server if called again.

---

In [4]:
# Request scene with the associated server clients
if not server.request_scene():
    sys.exit() # Scene request failed!

[SERVER] Wrapping up CoppeliaSim scene info
[SERVER] Scene info sent
[SERVER] Scene set!


# `Synchronizer` Structure

The `Synchronizer` class is a data structure that will be associated with a client during a capture process. It will store data sent by it's associated client and synchronize it by interpolating the position of the detected blobs in a standard time vector.

When instanciated, this data structure will need to be given:
- The number of expected blobs in the capture;
- The number of messages needed for interpolation - it's interpolation window;
- The interpolation step;
- The total capture time.

Notice that some of the parameters listed can restrict the overall behaviour of the capture. Parameters that can change dynamically are preferred for a more robust and versatile system, but it can also make it more complex to code.

The insertion of the data will be done by the `add_data` function. It will receive the undistorted blobs that the client detected and the presentation time stamp (PTS) that it was captured. It then returns a boolean flag that signals if the data was accepted or refused.

Then, the following actions must be taken:
1. Validate the data;
    - Check if the message is taken outside of the capture time range;
    - Check if the new message comes after the last one;
2. Order blobs - make sure that the interpolation done is from the same blob;  
    - This is done by ordering the blobs by their proximity to the blobs to the previous image;
    - The funcion `proximity_order` does this using the Munkres assignment algorithm;
3. Add blobs and their respective timestamps to the data structure;
4. If interpolation is possible, interpolate the yet to be interpolated data.

---

In [5]:
def proximity_order(previous_blobs, current_blobs):    
    distance_matrix = np.array([[np.linalg.norm(previous_blob - current_blob) 
                                for current_blob in current_blobs] 
                                for previous_blob in previous_blobs])

    # Using the hungarian (Munkres) assignment algorithm to find unique correspondences between blobs
    _, new_indices = linear_sum_assignment(distance_matrix)

    return current_blobs[new_indices]

In [6]:
# Data structure for data interpolation
class Synchronizer:
    def __init__(self, 
                 blob_count=1, # Number of expected blobs for interpolation
                 window=3,  # The minimum ammount of data points for interpolating 
                 step=0.05, # Time step for interpolation in seconds
                 capture_time=10 # Capture time in seconds
                 ):
        
        # Initializing interpolation parameters
        self.blob_count = blob_count
        self.interpolation_window = window
        self.step = step
        self.capture_time = capture_time
        self.interpolation_start = 0

        # Raw data - how it comes from the clients
        self.async_PTS = []
        self.async_blobs = []

        # Interpolated data - how it should be triangulated
        self.sync_PTS = np.arange(0.0, self.capture_time, self.step)
        self.sync_blobs = np.full((self.sync_PTS.size, blob_count, 2), -1.0) # Non-interpolated blobs are negative
       
    def add_data(self, blobs, PTS):
        # Do not add data if PTS is out of recording range
        if PTS > self.capture_time:
            return False # Data refused

        # Check if it's not empty
        if self.async_PTS:
            # Do not add data if incoming PTS is lesser or equal than the last PTS added
            # - Assure strictly ascending order of time
            # - Avoid sequenced repeated messaging
            if PTS <= self.async_PTS[-1]:
                return False # Data refused

        # Ordering blobs of this message by their proximity to the others on the previous message 
        if self.async_blobs: # If there are blobs
            # Try to order blobs
            ordered_blobs = proximity_order(self.async_blobs[-1], blobs)

            self.async_blobs.append(ordered_blobs)
            self.async_PTS.append(PTS)

        else: # If there aren't any blobs, just add them
            self.async_blobs.append(blobs)
            self.async_PTS.append(PTS)

        # Enough points to interpolate in the same blob ordering?
        if len(self.async_PTS) >= self.interpolation_window:
            # Get the window last blob coordinate data
            async_PTS = np.array(self.async_PTS[-self.interpolation_window:])
            async_blobs = np.array(self.async_blobs[-self.interpolation_window:])
            
            start = self.interpolation_start    # Start index of interpolated PTS  
            end = int(async_PTS[-1] // self.step) # Final index of interpolated PTS  

            for blob in range(self.blob_count):
                # Extracting blob coordinate history in the interpolation window
                # The slicing works like: async_blob = async_blobs[PTS, blob, axis]
                async_blob = async_blobs[:, blob, :]

                # Generating a cubic spline that represents blob trajectory 
                blob_trajectory = CubicSpline(async_PTS, async_blob)
                
                # Get blob tracjectory in the interpolated timestamps that are not yet interpolated
                interpolated_blob = blob_trajectory(self.sync_PTS[start:end+1]) # End of slice is exclusive!

                # Add interpolated blobs to data structure
                self.sync_blobs[start:end+1, blob, :] = interpolated_blob

            # Updating interpolation start to the next PTS 
            self.interpolation_start = end + 1

        return True # Data accepted

# Requesting Capture

A **Capture Request** will trigger a simulation in *CoppeliaSim* sending the total simulation time of the requested capture. Once the simulation stops, another capture request can be called for another simulation

In the simulation start, the clients will be created and send their ID to the server for client registration.

---

In [7]:
# Capture specifications
blob_count = 4 # Number of expected markers
capture_time = 10.0 # In seconds
window = 3 # The minimum ammount of points for interpolating 
throughput = 20 # Triangulated scenes per second
step = 1 / throughput # Interpolation timestep

# Capture synchronizer
synchronizer = Synchronizer(blob_count, window, step, capture_time)

# Request capture (start simulation)
if not server.request_capture(synchronizer):
    sys.exit() # Capture request failed!

# Wait for client identification
server.register_clients()

[SERVER] Capture info sent
[SERVER] Capture confirmed!
[SERVER] Waiting for clients...
	Client 0 registered
	Client 1 registered
	Client 2 registered
	Client 3 registered
[SERVER] All clients registered!


# Running Simulation

The messages will be received here in the following loop until a server timeout is reached. To analyse the content of each message, toggle the `verbose` flag. 

The loop will wait for a message to be received by the socket. When a message comes, the following actions will be taken:
1. Check for if timeout was reached and terminate close the loop if that's the case;
2. Identify which client is the message from;
3. Decode message; 
4. Parse the message for it's contents;
5. Check if the message is valid;
    - A valid message is composed of a blob coodinate and it's area (per blob) and the PTS of the message.
6. Save data in the `Synchronizer` structure.

---

In [8]:
verbose = False

timeout = 5 # In seconds
server.udp_socket.settimeout(timeout) # Set server timeout
print(f'[SERVER] Timeout set to {timeout} seconds\n')

# Breaks in the timeout
while True: 
    # Wait for message - Event guided!
    try:
        message_bytes, address = server.udp_socket.recvfrom(server.buffer_size)

    except socket.timeout as err:
        print('\n[SERVER] Timed Out!')
        
        break # Close capture loop due to timeout
    
    # Check if client exists
    try:
        ID = server.client_addresses[address] # Client Identifier
    
    except:
        if verbose: print('> Client not recognized')

        continue # Jump to wait for the next message
    
    # Show sender
    if verbose: print(f'> Received message from Client {ID} ({address[0]}, {address[1]}):')

    # Decode message
    try:
        message = np.frombuffer(message_bytes, dtype=np.float32)

    except:
        if verbose: print('> Couldn\'t decode message')

        continue # Jump to wait for the next message

    # Empty message
    if not message.size:
        if verbose: print('\tEmpty message')

        continue # Jump to wait for the next message

    # Extracting the message's PTS
    PTS = message[-1] # Last element of the message 

    # Valid message is [u, v, A] per blob and the PTS of the message
    if message.size !=  3 * blob_count + 1:

        if message.size == 1: # Only PTS
            if verbose: print(f'\tNo blobs were detected - {PTS :.3f} s')

        else: 
            if verbose: 
                print(f'\tWrong blob count or corrupted message')
                print(f'Corrupted Message: {message}')

        continue # Jump to wait for the next message

    # Extracting blob data (coordinates & area)
    blob_data = message[:-1].reshape(-1, 3) # All but last element (reserved for PTS)

    # Extracting centroids
    blob_centroids = blob_data[:,:2] # Ignoring their area

    # Undistorting blobs centroids
    undistorted_blobs = server.clients[ID].camera.undistort_points(blob_centroids)          

    # Print blobs
    if verbose:
        print(f'\tDetected Blobs - {PTS :.3f} s')
        print('\t' + str(blob_data).replace('\n', '\n\t'))

    # Save data
    valid_data = server.clients[ID].synchronizer.add_data(undistorted_blobs, PTS)

    if verbose: 
        if valid_data:
            print('\tData Accepted!')
        else:
            print('\tData Refused!')

[SERVER] Timeout set to 5 seconds


[SERVER] Timed Out!


# Playing Back a Camera's Feed

The following cell will replicate a real time camera feed of the simulation. Change the `ID` parameter to switch between camera views.

---

In [13]:
# Playback a camera image feed in fidelity time
ID = 0 # Camera ID to be watched
synchronizer = server.clients[ID].synchronizer
camera = server.clients[ID].camera

for (PTS_data, blob_data, type) in [(synchronizer.async_PTS, synchronizer.async_blobs, 'Asynchronous'), 
                                    (synchronizer.sync_PTS,  synchronizer.sync_blobs,  'Synchronous')]:
    
    # Converting to np arrays
    PTS_data = np.array(PTS_data)
    blob_data = np.array(blob_data)
    
    # Generating frames
    images = []
    for PTS, blobs in zip(PTS_data, blob_data):
        image = np.zeros(camera.resolution)

        for blob in blobs:
            image = cv2.circle(image, blob.astype(int), 2, 1, -1) 
            
        images.append(image)

    # Getting delay between each frame
    delays = PTS_data[1:] - PTS_data[:-1]

    # Playing the animation
    for delay, image in zip(delays, images):
        cv2.imshow(f'Camera {ID} - {type}' , image)
        cv2.waitKey(int(1e3 * delay))

    # Closing all open windows 
    cv2.destroyAllWindows()